# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

In [1]:
# Import your libraries:

%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In this lab, we will explore a dataset that describes websites with different features and labels them either benign or malicious . We will use supervised learning algorithms to figure out what feature patterns malicious websites are likely to have and use our model to predict malicious websites.

# Challenge 1 - Explore The Dataset

Let's start by exploring the dataset. First load the data file:

In [2]:
websites = pd.read_csv('../website.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../website.csv'

#### Explore the data from an bird's-eye view.

You should already been very familiar with the procedures now so we won't provide the instructions step by step. Reflect on what you did in the previous labs and explore the dataset.

Things you'll be looking for:

* What the dataset looks like?
* What are the data types?
* Which columns contain the features of the websites?
* Which column contains the feature we will predict? What is the code standing for benign vs malicious websites?
* Do we need to transform any of the columns from categorical to ordinal values? If so what are these columns?

Feel free to add additional cells for your explorations. Make sure to comment what you find out.

In [ ]:
# Your code here
websites.dtypes

In [ ]:
# Your comment here
websites.head()

#### Next, evaluate if the columns in this dataset are strongly correlated.

In the Mushroom supervised learning lab we did recently, we mentioned we are concerned if our dataset has strongly correlated columns because if it is the case we need to choose certain ML algorithms instead of others. We need to evaluate this for our dataset now.

Luckily, most of the columns in this dataset are ordinal which makes things a lot easier for us. In the next cells below, evaluate the level of collinearity of the data.

We provide some general directions for you to consult in order to complete this step:

1. You will create a correlation matrix using the numeric columns in the dataset.

1. Create a heatmap using `seaborn` to visualize which columns have high collinearity.

1. Comment on which columns you might need to remove due to high collinearity.

In [ ]:
# Your code here
websites.corr()

In [ ]:
corr = websites.corr()
f, ax = plt.subplots(figsize=(15, 15))
mask = np.triu(np.ones_like(corr, dtype=bool)) # removing the other side of the heatmap
cmap = sns.diverging_palette(230, 20, as_cmap=True) #preparing cmap
sns.heatmap(corr,mask=mask,cmap=cmap,linewidths=.5,square=True,annot=True)
plt.show()


In [ ]:
# identifying columns with  correlation above 90%

# Create correlation matrix
corr_matrix = websites.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.91)]
to_drop


In [ ]:
# Your comment here
# drop remote_app_packets, source_app_bytes, TCP, App_Bytes

# Challenge 2 - Remove Column Collinearity.

From the heatmap you created, you should have seen at least 3 columns that can be removed due to high collinearity. Remove these columns from the dataset.

Note that you should remove as few columns as you can. You don't have to remove all the columns at once. But instead, try removing one column, then produce the heatmap again to determine if additional columns should be removed. As long as the dataset no longer contains columns that are correlated for over 90%, you can stop. Also, keep in mind when two columns have high collinearity, you only need to remove one of them but not both.

In the cells below, remove as few columns as you can to eliminate the high collinearity in the dataset. Make sure to comment on your way so that the instructional team can learn about your thinking process which allows them to give feedback. At the end, print the heatmap again.

In [ ]:
# Your code here
df = websites.drop(columns=to_drop)
df.head()

In [ ]:
# Your comment here

In [ ]:
# Print heatmap again
corr = df.corr()
f, ax = plt.subplots(figsize=(15, 15))
mask = np.triu(np.ones_like(corr, dtype=bool)) # removing the other side of the heatmap
cmap = sns.diverging_palette(230, 20, as_cmap=True) #preparing cmap
sns.heatmap(corr,mask=mask,cmap=cmap,linewidths=.5,square=True,annot=True)
plt.show()


# Challenge 3 - Handle Missing Values

The next step would be handling missing values. **We start by examining the number of missing values in each column, which you will do in the next cell.**

In [ ]:
# Your code here
display(df.isnull().sum())
display(df.shape)

If you remember in the previous labs, we drop a column if the column contains a high proportion of missing values. After dropping those problematic columns, we drop the rows with missing values.

#### In the cells below, handle the missing values from the dataset. Remember to comment the rationale of your decisions.

In [ ]:
# Your code here
df.drop("CONTENT_LENGTH", inplace=True, axis=1)

In [ ]:
# Your comment here
# because there are 812 missing values

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

#### Again, examine the number of missing values in each column. 

If all cleaned, proceed. Otherwise, go back and do more cleaning.

In [ ]:
# Examine missing values in each column


# Challenge 4 - Handle `WHOIS_*` Categorical Data

There are several categorical columns we need to handle. These columns are:

* `URL`
* `CHARSET`
* `SERVER`
* `WHOIS_COUNTRY`
* `WHOIS_STATEPRO`
* `WHOIS_REGDATE`
* `WHOIS_UPDATED_DATE`

How to handle string columns is always case by case. Let's start by working on `WHOIS_COUNTRY`. Your steps are:

1. List out the unique values of `WHOIS_COUNTRY`.
1. Consolidate the country values with consistent country codes. For example, the following values refer to the same country and should use consistent country code:
    * `CY` and `Cyprus`
    * `US` and `us`
    * `SE` and `se`
    * `GB`, `United Kingdom`, and `[u'GB'; u'UK']`

#### In the cells below, fix the country values as intructed above.

In [ ]:
# Your code here
# df["WHOIS_COUNTRY"].value_counts()
df2 = df.copy()

pd.DataFrame(df2["WHOIS_COUNTRY"].value_counts()).reset_index()


In [ ]:
df2['WHOIS_COUNTRY'] = df2['WHOIS_COUNTRY'].apply(lambda x: x.replace("us", "US"))
df2['WHOIS_COUNTRY'] = df2['WHOIS_COUNTRY'].apply(lambda x: x.replace("CyprUS", "CY"))
df2['WHOIS_COUNTRY'] = df2['WHOIS_COUNTRY'].apply(lambda x: x.replace("se", "SE"))
df2['WHOIS_COUNTRY'] = df2['WHOIS_COUNTRY'].apply(lambda x: x.replace("United Kingdom", "GB"))
df2['WHOIS_COUNTRY'] = df2['WHOIS_COUNTRY'].apply(lambda x: x.replace("[u'GB'; u'UK']", "GB"))
df2['WHOIS_COUNTRY'] = df2['WHOIS_COUNTRY'].apply(lambda x: x.replace("ru", "RU"))
pd.DataFrame(df2["WHOIS_COUNTRY"].value_counts()).reset_index()

In [ ]:
df2['WHOIS_COUNTRY'] = np.where(df2['WHOIS_COUNTRY'] == "[u'GB'; u'UK']",'UK',df2['WHOIS_COUNTRY'])
df2['WHOIS_COUNTRY'] = np.where(df2['WHOIS_COUNTRY'] == "Cyprus",'CY',df2['WHOIS_COUNTRY'])
pd.DataFrame(df2["WHOIS_COUNTRY"].value_counts()).reset_index()

In [ ]:
df2["WHOIS_COUNTRY"]=df2["WHOIS_COUNTRY"].str.replace("Cyprus", "CY").replace("United Kingdom", "UK")
pd.DataFrame(df2["WHOIS_COUNTRY"].value_counts()).reset_index()

Since we have fixed the country values, can we convert this column to ordinal now?

Not yet. If you reflect on the previous labs how we handle categorical columns, you probably remember we ended up dropping a lot of those columns because there are too many unique values. Too many unique values in a column is not desirable in machine learning because it makes prediction inaccurate. But there are workarounds under certain conditions. One of the fixable conditions is:

#### If a limited number of values account for the majority of data, we can retain these top values and re-label all other rare values.

The `WHOIS_COUNTRY` column happens to be this case. You can verify it by print a bar chart of the `value_counts` in the next cell to verify:

In [ ]:
# Your code here
plt.figure(figsize=(15, 10))
df2['WHOIS_COUNTRY'].value_counts().plot(kind='bar')


#### After verifying, now let's keep the top 10 values of the column and re-label other columns with `OTHER`.

In [ ]:
# websites.loc[websites['WHOIS_COUNTRY'].value_counts()[websites['WHOIS_COUNTRY']].values < 10, 'WHOIS_COUNTRY'] = "OTHER"


In [ ]:
# Your code here
df2['WHOIS_COUNTRY'] = np.where(df2['WHOIS_COUNTRY'].isin(list(df2['WHOIS_COUNTRY'].value_counts().head(10).index)),df2['WHOIS_COUNTRY'],"Other")


In [ ]:
list(df2['WHOIS_COUNTRY'].value_counts().head(10).index)

Now since `WHOIS_COUNTRY` has been re-labelled, we don't need `WHOIS_STATEPRO` any more because the values of the states or provinces may not be relevant any more. We'll drop this column.

In addition, we will also drop `WHOIS_REGDATE` and `WHOIS_UPDATED_DATE`. These are the registration and update dates of the website domains. Not of our concerns.

#### In the next cell, drop `['WHOIS_STATEPRO', 'WHOIS_REGDATE', 'WHOIS_UPDATED_DATE']`.

In [ ]:
# Your code here
# df2.drop(columns=['WHOIS_STATEPRO', 'WHOIS_REGDATE', 'WHOIS_UPDATED_DATE',"category"], inplace=True)
df2.head()

# Challenge 5 - Handle Remaining Categorical Data & Convert to Ordinal

Now print the `dtypes` of the data again. Besides `WHOIS_COUNTRY` which we already fixed, there should be 3 categorical columns left: `URL`, `CHARSET`, and `SERVER`.

In [ ]:
# Your code here
df2.dtypes

#### `URL` is easy. We'll simply drop it because it has too many unique values that there's no way for us to consolidate.

In [ ]:
# Your code here
df2.drop(columns="URL", inplace=True)

#### Print the unique value counts of `CHARSET`. You see there are only a few unique values. So we can keep it as it is.

In [ ]:
# Your code here
df2["CHARSET"].unique()

`SERVER` is a little more complicated. Print its unique values and think about how you can consolidate those values.

#### Before you think of your own solution, don't read the instructions that come next.

In [ ]:
# Your code here
df2["SERVER"].value_counts()

![Think Hard](../think-hard.jpg)

In [ ]:
# Your comment here


Although there are so many unique values in the `SERVER` column, there are actually only 3 main server types: `Microsoft`, `Apache`, and `nginx`. Just check if each `SERVER` value contains any of those server types and re-label them. For `SERVER` values that don't contain any of those substrings, label with `Other`.

At the end, your `SERVER` column should only contain 4 unique values: `Microsoft`, `Apache`, `nginx`, and `Other`.

In [ ]:
# Your code here
df2['SERVER'] = np.where(df2['SERVER'].str.contains('Microsoft'), "Microsoft", df2['SERVER'])
df2['SERVER'] = np.where(df2['SERVER'].str.contains('Apache'), "Apache", df2['SERVER'])
df2['SERVER'] = np.where(df2['SERVER'].str.contains('nginx'), "nginx", df2['SERVER'])
df2['SERVER'] = np.where(df2['SERVER'].isin(["Microsoft","Apache","nginx"]),df2["SERVER"],"Other")


In [ ]:
# From Kate another soluition, please change the dataframe name
# websites['SERVER'][websites['SERVER'].str.contains('Microsoft')==True] = 'Microsoft'
# websites['SERVER'][websites['SERVER'].str.contains('Apache')==True] = 'Apache'
# websites['SERVER'][websites['SERVER'].str.contains('nginx')==True] = 'nginx'
# websites['SERVER'][websites['SERVER'].str.contains('Microsoft|Apache|nginx')==False] = 'Other'


In [ ]:
# Count `SERVER` value counts here
df2["SERVER"].value_counts()

OK, all our categorical data are fixed now. **Let's convert them to ordinal data using Pandas' `get_dummies` function ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)).** Make sure you drop the categorical columns by passing `drop_first=True` to `get_dummies` as we don't need them any more. **Also, assign the data with dummy values to a new variable `website_dummy`.**

In [ ]:
# Your code here
df2.head()

In [ ]:
df2.shape

Now, inspect `website_dummy` to make sure the data and types are intended - there shouldn't be any categorical columns at this point.

In [ ]:
# Your code here
website_dummy = pd.get_dummies(df2, columns=["CHARSET", "SERVER", "WHOIS_COUNTRY"], drop_first=True)
website_dummy.head()

# Challenge 6 - Modeling, Prediction, and Evaluation

We'll start off this section by splitting the data to train and test. **Name your 4 variables `X_train`, `X_test`, `y_train`, and `y_test`. Select 80% of the data for training and 20% for testing.**

In [ ]:
from sklearn.model_selection import train_test_split
X = website_dummy.drop(columns="Type")
y = website_dummy["Type"]
# Your code here:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

#### In this lab, we will try two different models and compare our results.

The first model we will use in this lab is logistic regression. We have previously learned about logistic regression as a classification algorithm. In the cell below, load `LogisticRegression` from scikit-learn and initialize the model.

In [ ]:
# Your code here:
from sklearn.linear_model import LogisticRegression
website_model = LogisticRegression().fit(X_train, y_train)


Next, fit the model to our training data. We have already separated our data into 4 parts. Use those in your model.

In [ ]:
# Your code here:
# See above


finally, import `confusion_matrix` and `accuracy_score` from `sklearn.metrics` and fit our testing data. Assign the fitted data to `y_pred` and print the confusion matrix as well as the accuracy score

In [ ]:
# Your code here:
from sklearn.metrics import confusion_matrix
y_pred = website_model.predict(X_test)
cf_matrix = confusion_matrix(y_test, y_pred)
print(cf_matrix)

In [ ]:
pd.Series(y_pred).value_counts()

In [ ]:
pd.Series(y_test).value_counts()

In [ ]:
# 83% website are begnin that we said they are begnin
# positive means  malicious what is the bad thing such as credit problem
# 1.9% website are malicious that we said they are begnin

In [ ]:
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
print(precision_score(y_test, y_pred, pos_label=1))
print(recall_score(y_test, y_pred, pos_label=1))
print(f1_score(y_test, y_pred, pos_label=1))

What are your thoughts on the performance of the model? Write your conclusions below.

In [ ]:
# Your conclusions here:



#### Our second algorithm is is K-Nearest Neighbors. 

Though is it not required, we will fit a model using the training data and then test the performance of the model using the testing data. Start by loading `KNeighborsClassifier` from scikit-learn and then initializing and fitting the model. We'll start off with a model where k=3.

In [ ]:
# Your code here:



To test your model, compute the predicted values for the testing sample and print the confusion matrix as well as the accuracy score.

In [ ]:
# Your code here:



#### We'll create another K-Nearest Neighbors model with k=5. 

Initialize and fit the model below and print the confusion matrix and the accuracy score.

In [ ]:
# Your code here:



Did you see an improvement in the confusion matrix when increasing k to 5? Did you see an improvement in the accuracy score? Write your conclusions below.

In [ ]:
# Your conclusions here:



# Bonus Challenge - Feature Scaling

Problem-solving in machine learning is iterative. You can improve your model prediction with various techniques (there is a sweetspot for the time you spend and the improvement you receive though). Now you've completed only one iteration of ML analysis. There are more iterations you can conduct to make improvements. In order to be able to do that, you will need deeper knowledge in statistics and master more data analysis techniques. In this bootcamp, we don't have time to achieve that advanced goal. But you will make constant efforts after the bootcamp to eventually get there.

However, now we do want you to learn one of the advanced techniques which is called *feature scaling*. The idea of feature scaling is to standardize/normalize the range of independent variables or features of the data. This can make the outliers more apparent so that you can remove them. This step needs to happen during Challenge 6 after you split the training and test data because you don't want to split the data again which makes it impossible to compare your results with and without feature scaling. For general concepts about feature scaling, click [here](https://en.wikipedia.org/wiki/Feature_scaling). To read deeper, click [here](https://medium.com/greyatom/why-how-and-when-to-scale-your-features-4b30ab09db5e).

In the next cell, attempt to improve your model prediction accuracy by means of feature scaling. A library you can utilize is `sklearn.preprocessing.RobustScaler` ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)). You'll use the `RobustScaler` to fit and transform your `X_train`, then transform `X_test`. You will use logistic regression to fit and predict your transformed data and obtain the accuracy score in the same way. Compare the accuracy score with your normalized data with the previous accuracy data. Is there an improvement?

In [ ]:
# Your code here